In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top-500-indian-cities/cities_r2.csv
/kaggle/input/reddit-scrape-lockdown/pre_lockdown_reddit.csv
/kaggle/input/reddit-scrape-lockdown/lockdown_reddit.csv


In [ ]:
!pip install praw

In [ ]:
import praw
import pandas as pd
import datetime as dt

In [ ]:
# Creating a reddit crawler object 
reddit = praw.Reddit(client_id='ISXAFnIORvSYVw',
                     client_secret='aZByxgoktINt9snO4UyISmsjULE',
                     user_agent='covid19',
                     username='MohammadAmaan29',
                     password='faheemamaan1')

In [ ]:
# Creating and crawling a particular subreddit

subreddit = reddit.subreddit('india')
for post in subreddit.search('abuse',limit=500):
    print(post.title)
topics_dict = {"title":[],
               "score":[],
               "id":[], "url":[],
               "comms_num": [],
               "created": [],
               "body":[]}
for submission in subreddit.search('abuse',limit=100):
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    
abuse_data = pd.DataFrame(topics_dict)
def get_date(created):
    return dt.datetime.fromtimestamp(created)
_timestamp = abuse_data["created"].apply(get_date)
abuse_data = abuse_data.assign(timestamp = _timestamp)
abuse_data

# Subreddit Object

We will now create a custom scrape_subreddit object that will help us group all the functions and scrape a subreddit

In [ ]:
class scrape_subreddit:
    def __init__(self,reddit,subred,tag,no_of_results):
        self.name = subred
        self.dataframe = None
        self.subreddit = reddit.subreddit(subred)
        self.tag = tag
        self.no_of_results=no_of_results
        self.topics_dict = {"title":[],
               "score":[],
               "id":[], "url":[],
               "comms_num": [],
               "created": [],
               "body":[]}
        
    def scrape_data(self):
        for submission in self.subreddit.search(self.tag,limit=self.no_of_results):
            self.topics_dict["title"].append(submission.title)
            self.topics_dict["score"].append(submission.score)
            self.topics_dict["id"].append(submission.id)
            self.topics_dict["url"].append(submission.url)
            self.topics_dict["comms_num"].append(submission.num_comments)
            self.topics_dict["created"].append(submission.created)
            self.topics_dict["body"].append(submission.selftext)
        return self.topics_dict
    
    def get_dataframe(self):
        self.dataframe = pd.DataFrame(self.scrape_data())
        self.temp_timestamp = self.dataframe["created"].apply(lambda x : dt.datetime.fromtimestamp(x))
        self.dataframe = self.dataframe.assign(timestamp = self.temp_timestamp)
        self.dataframe['year'] = self.dataframe['timestamp'].apply(lambda x : x.year)
        self.dataframe['month'] = self.dataframe['timestamp'].apply(lambda x : x.month)
        self.dataframe['day'] = self.dataframe['timestamp'].apply(lambda x : x.day)
        self.dataframe['time'] = self.dataframe['timestamp'].apply(lambda x : x.time())
        self.dataframe.drop(['timestamp'],axis=1,inplace=True)
        self.dataframe = self.dataframe.drop_duplicates(subset='title',keep="first")
        return self.dataframe
    
    def get_lockdown_data(self):
        self.dataframe = self.get_dataframe()
        self.dataframe = self.dataframe.loc[(self.dataframe['year']==2020) & (self.dataframe['month']>2),:]
        self.dataframe.reset_index(inplace=True)
        return self.dataframe
    
    def get_pre_lockdown_data(self):
        self.dataframe = self.get_dataframe()
        self.dataframe = self.dataframe.loc[(self.dataframe['year']<=2020) & (self.dataframe['month']<3),:]
        self.dataframe.reset_index(inplace=True)
        return self.dataframe
            
                

In [ ]:
def scrape(reddit,subreddit,tag,limit):
    reddit = reddit
    scrape_object = scrape_subreddit(reddit,subreddit,tag,limit)
    pre_lockdown_data = scrape_object.get_pre_lockdown_data()
    lockdown_data = scrape_object.get_lockdown_data()
    return pre_lockdown_data,lockdown_data

# Scraping the Subreddits

## 1. r/india

### We'll scrape the subreddit r/india since we only need indian data

### We'll also scrape popular subreddits like r/relationship_advice,r/relationships and r/relationships

## 2. Other subreddits
### Since some subreddits are specific to violence types, we'll scrape them using the keyword 'india'

## r/india

### 1. Abuse

In [ ]:
abuse_pre_data,abuse_lockdown_data = scrape(reddit,'india','abuse',500)

abuse_pre_data['violence_type'] = ['abuse']*len(abuse_pre_data)
abuse_lockdown_data['violence_type'] = ['abuse']*len(abuse_lockdown_data)
pre_data = abuse_pre_data
lockdown_data = abuse_lockdown_data

print(f"Number of data points in abuse_pre_data i.e, before lockdown : {len(abuse_pre_data)}")
print(f"Number of data points in abuse_lockdown_data : {len(abuse_lockdown_data)}")

In [ ]:
abuse_pre_data2,abuse_lockdown_data2 = scrape(reddit,'relationship_advice','india+abuse',500)

abuse_pre_data2['violence_type'] = ['abuse']*len(abuse_pre_data2)
abuse_lockdown_data2['violence_type'] = ['abuse']*len(abuse_lockdown_data2)


print(f"Number of data points in abuse_pre_data2 i.e, before lockdown : {len(abuse_pre_data2)}")
print(f"Number of data points in abuse_lockdown_data2 : {len(abuse_lockdown_data2)}")

In [ ]:
abuse_pre_data3,abuse_lockdown_data3 = scrape(reddit,'relationships','india abuse',500)

abuse_pre_data3['violence_type'] = ['abuse']*len(abuse_pre_data3)
abuse_lockdown_data3['violence_type'] = ['abuse']*len(abuse_lockdown_data3)


print(f"Number of data points in abuse_pre_data3 i.e, before lockdown : {len(abuse_pre_data3)}")
print(f"Number of data points in abuse_lockdown_data3 : {len(abuse_lockdown_data3)}")

In [ ]:
abuse_pre_data4,abuse_lockdown_data4 = scrape(reddit,'advice','india abuse',500)

abuse_pre_data4['violence_type'] = ['abuse']*len(abuse_pre_data4)
abuse_lockdown_data4['violence_type'] = ['abuse']*len(abuse_lockdown_data4)


print(f"Number of data points in abuse_pre_data4 i.e, before lockdown : {len(abuse_pre_data4)}")
print(f"Number of data points in abuse_lockdown_data4 : {len(abuse_lockdown_data4)}")

### 2.Dowry

In [ ]:
dowry_pre_data,dowry_lockdown_data = scrape(reddit,'india','dowry',500)

dowry_pre_data['violence_type'] = ['dowry']*len(dowry_pre_data)
dowry_lockdown_data['violence_type'] = ['dowry']*len(dowry_lockdown_data)

print(f"Number of data points in dowry_pre_data : {len(dowry_pre_data)}")
print(f"Number of data points in dowry_lockdown_data : {len(dowry_lockdown_data)}")

In [ ]:
dowry_pre_data2,dowry_lockdown_data2 = scrape(reddit,'relationship_advice','india dowry',500)

dowry_pre_data2['violence_type'] = ['dowry']*len(dowry_pre_data2)
dowry_lockdown_data2['violence_type'] = ['dowry']*len(dowry_lockdown_data2)

print(f"Number of data points in dowry_pre_data2 : {len(dowry_pre_data2)}")
print(f"Number of data points in dowry_lockdown_data2 : {len(dowry_lockdown_data2)}")

In [ ]:
dowry_pre_data3,dowry_lockdown_data3 = scrape(reddit,'advice','india dowry',500)

dowry_pre_data3['violence_type'] = ['dowry']*len(dowry_pre_data3)
dowry_lockdown_data3['violence_type'] = ['dowry']*len(dowry_lockdown_data3)

print(f"Number of data points in dowry_pre_data3 : {len(dowry_pre_data3)}")
print(f"Number of data points in dowry_lockdown_data3 : {len(dowry_lockdown_data3)}")

### Rape

In [ ]:
rape_pre_data,rape_lockdown_data = scrape(reddit,'india','rape',500)

rape_pre_data['violence_type'] = ['rape']*len(rape_pre_data)
rape_lockdown_data['violence_type'] = ['rape']*len(rape_lockdown_data)

print(f"Number of data points in rape_pre_data {len(rape_pre_data)}")
print(f"Number of data points in rape_lockdown_data {len(rape_lockdown_data)}")

In [ ]:
rape_pre_data2,rape_lockdown_data2 = scrape(reddit,'relationship_advice','india rape',500)

rape_pre_data2['violence_type'] = ['rape']*len(rape_pre_data2)
rape_lockdown_data2['violence_type'] = ['rape']*len(rape_lockdown_data2)

print(f"Number of data points in rape_pre_data2 {len(rape_pre_data2)}")
print(f"Number of data points in rape_lockdown_data2 {len(rape_lockdown_data2)}")

### Justice for 

In [ ]:
justice_pre_data,justice_lockdown_data = scrape(reddit,'india','justice for',500)

justice_pre_data['violence_type'] = ['justice']*len(justice_pre_data)
justice_lockdown_data['violence_type'] = ['justice'] * len(justice_lockdown_data)

print(f"Number of data points in justice_pre_data {len(justice_pre_data)}")
print(f"Number of data points in justice_lockdown_data {len(justice_lockdown_data)}")

### Robberies

In [ ]:
robbery_pre_data,robbery_lockdown_data = scrape(reddit,'india','robbery',500)

robbery_pre_data['violence_type'] = ['robbery']* len(robbery_pre_data)
robbery_lockdown_data['violence_type'] = ['robbery']*len(robbery_lockdown_data)

print(f"Number of data points in roberry_pre_data {len(robbery_pre_data)}")
print(f"Number of data points in roberry_lockdown_data {len(robbery_lockdown_data)}")

### Harassment

In [ ]:
harassment_pre_data,harassment_lockdown_data = scrape(reddit,'india','harassment',1000)

harassment_pre_data['violence_type'] = ['harassment']* len(harassment_pre_data)
harassment_lockdown_data['violence_type'] = ['harassment']*len(harassment_lockdown_data)

print(f"Number of data points in harassment_pre_data {len(harassment_pre_data)}")
print(f"Number of data points in harassment_lockdown_data {len(harassment_lockdown_data)}")

In [ ]:
harassment_pre_data2,harassment_lockdown_data2 = scrape(reddit,'relationship_advice','india harassment',1000)

harassment_pre_data2['violence_type'] = ['harassment']* len(harassment_pre_data2)
harassment_lockdown_data2['violence_type'] = ['harassment']*len(harassment_lockdown_data2)

print(f"Number of data points in harassment_pre_data {len(harassment_pre_data2)}")
print(f"Number of data points in harassment_lockdown_data {len(harassment_lockdown_data2)}")

### Survivor

In [ ]:
survivor_pre_data,survivor_lockdown_data = scrape(reddit,'india','survivor',1000)

survivor_pre_data['violence_type'] = ['survivor']* len(survivor_pre_data)
survivor_lockdown_data['violence_type'] = ['survivor']*len(survivor_lockdown_data)

print(f"Number of data points in survivor_pre_data {len(survivor_pre_data)}")
print(f"Number of data points in survivor_lockdown_data {len(survivor_lockdown_data)}")

### victim

In [ ]:
victim_pre_data,victim_lockdown_data = scrape(reddit,'india','victim',1000)

victim_pre_data['violence_type'] = ['victim']* len(victim_pre_data)
victim_lockdown_data['violence_type'] = ['victim']*len(victim_lockdown_data)

print(f"Number of data points in victim_pre_data {len(victim_pre_data)}")
print(f"Number of data points in victim_lockdown_data {len(victim_lockdown_data)}")

In [ ]:
victim_pre_data2,victim_lockdown_data2 = scrape(reddit,'relationship_advice','india victim',1000)

victim_pre_data2['violence_type'] = ['victim']* len(victim_pre_data2)
victim_lockdown_data2['violence_type'] = ['victim']*len(victim_lockdown_data2)

print(f"Number of data points in victim_pre_data {len(victim_pre_data2)}")
print(f"Number of data points in victim_lockdown_data {len(victim_lockdown_data2)}")

# Merging data

In [ ]:
pre_data = pd.concat([abuse_pre_data,abuse_pre_data2,abuse_pre_data3,abuse_pre_data4,
                      dowry_pre_data,dowry_pre_data2,dowry_pre_data3,rape_pre_data,rape_pre_data2,
                      justice_pre_data,robbery_pre_data,harassment_pre_data,harassment_pre_data2,
                      survivor_pre_data,victim_pre_data])

lockdown_data = pd.concat([abuse_lockdown_data,abuse_lockdown_data2,abuse_lockdown_data3,
                           abuse_lockdown_data4,dowry_lockdown_data,dowry_lockdown_data2,
                           dowry_lockdown_data3,rape_lockdown_data,rape_lockdown_data2,
                           justice_lockdown_data,robbery_lockdown_data,harassment_lockdown_data,
                           harassment_lockdown_data2,survivor_lockdown_data,
                           victim_lockdown_data])

In [ ]:
print(f"Length of lockdown data {len(lockdown_data)}")
lockdown_data = lockdown_data.drop_duplicates(subset='title',keep='first')
print(f"Length of unique lockdown data {len(lockdown_data)}")

In [ ]:
print(f"Length of pre lockdown data {len(pre_data)}")
pre_data = pre_data.drop_duplicates(subset='title',keep='first')
print(f"Length of unique pre lockdown data {len(pre_data)}")

## 2. Other subreddits 

We'll focus on the following subreddits

1. raisedbynarcissists
2. adultsurvivors
3. legaladvice
4. Parenting
5. MensRights
6. abuse
7. GenderCritical

In [ ]:
multiple_subs_pre,multiple_subs_lockdown = scrape(reddit,'raisebynarcissists+adultsurvivors+legaladvice+Parenting+MensRights+abuse+GenderCritical'
                                                  ,'india',3000)
print(len(multiple_subs_pre))
print(len(multiple_subs_lockdown))

In [ ]:
pre_data = pd.concat([pre_data,multiple_subs_pre])
lockdown_data = pd.concat([lockdown_data,multiple_subs_lockdown])

In [ ]:
len(lockdown_data)

In [ ]:
lockdown_data.to_csv('lockdown_reddit.csv')
pre_data.to_csv('pre_lockdown_reddit.csv')

# Modelling

We'll extract the following features from the data also while filtering irrelevant entries parallely

* Age
* Gender
* Sexual Orientation
* Imagery
* Location
* Language
* Time of day when active
* relationship between victim and abuser


In [38]:
# Let's see what do we have

df = pd.read_csv('/kaggle/input/reddit-scrape-lockdown/lockdown_reddit.csv')

In [39]:
print(df.dtypes)
df.head()

Unnamed: 0       float64
index            float64
title             object
score             object
id                object
url               object
comms_num         object
created          float64
body              object
year             float64
month            float64
day              float64
time              object
violence_type     object
Unnamed: 14       object
dtype: object


,Unnamed: 0,index,title,score,id,url,comms_num,created,body,year,month,day,time,violence_type,Unnamed: 14
0,0.0,0.0,There is NO ROOM for abuse.Period. Digital Pai...,3114,gfoc94,https://i.redd.it/xfvcce15rhx41.jpg,329,1.588951e+09,NaN,2020.0,5.0,8.0,15:12:12,abuse,NaN
1,1.0,1.0,"Behind closed doors, the biggest viruses are m...",1128,fzzzji,https://i.redd.it/dc1v110v0fs41.jpg,213,1.586738e+09,NaN,2020.0,4.0,13.0,00:40:34,abuse,NaN
2,2.0,2.0,"Manipur girl racially attacked, abused by loca...",365,gici3u,https://newsd.in/manipur-girl-racially-attacke...,62,1.589323e+09,NaN,2020.0,5.0,12.0,22:43:00,abuse,NaN
3,3.0,3.0,Chennai man caught on camera hurling casteist ...,402,g30ho5,https://www.thenewsminute.com/article/chennai-...,114,1.587153e+09,NaN,2020.0,4.0,17.0,19:49:59,abuse,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Age

In [40]:
df['body'] = df['body'].fillna('None').apply(str)

In [41]:
import re
def age_extractor(sentence):
    match = re.findall(r"\b\d{2}(?!%)[^\d]", sentence)

    if match:
        return re.findall(r"\b\d{2}(?!%)[^\d]", sentence)[0][:2]
    else:
        return None


In [42]:
df['age'] = df['body'].apply(age_extractor)

In [43]:
df['age']

0      None
1      None
2      None
3      None
4      None
       ... 
532      16
533    None
534      15
535      10
536      30
Name: age, Length: 537, dtype: object

In [44]:
df.drop(['city','state'],axis=1,inplace=True)

KeyError: "['city' 'state'] not found in axis"

In [46]:
df = df.loc[df['title'].notna(),:]

In [47]:
len(df)

442

In [52]:
df

,Unnamed: 0,index,title,score,id,url,comms_num,created,body,year,month,day,time,violence_type,Unnamed: 14,age
0,0.0,0.0,There is NO ROOM for abuse.Period. Digital Pai...,3114,gfoc94,https://i.redd.it/xfvcce15rhx41.jpg,329,1.588951e+09,None,2020.0,5.0,8.0,15:12:12,abuse,NaN,None
1,1.0,1.0,"Behind closed doors, the biggest viruses are m...",1128,fzzzji,https://i.redd.it/dc1v110v0fs41.jpg,213,1.586738e+09,None,2020.0,4.0,13.0,00:40:34,abuse,NaN,None
2,2.0,2.0,"Manipur girl racially attacked, abused by loca...",365,gici3u,https://newsd.in/manipur-girl-racially-attacke...,62,1.589323e+09,None,2020.0,5.0,12.0,22:43:00,abuse,NaN,None
3,3.0,3.0,Chennai man caught on camera hurling casteist ...,402,g30ho5,https://www.thenewsminute.com/article/chennai-...,114,1.587153e+09,None,2020.0,4.0,17.0,19:49:59,abuse,NaN,None
5,5.0,5.0,Parental Abuse: What to do?,31,ggfu6u,https://www.reddit.com/r/india/comments/ggfu6u...,49,1.589061e+09,I am writing this with such a heavy heart. But...,2020.0,5.0,9.0,21:56:50,abuse,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,117.0,228.0,Do the parties signing a legal contract have c...,1,gbeobj,https://www.reddit.com/r/legaladvice/comments/...,0,1.588352e+09,I am a 16 yo developer and I have an idea abou...,2020.0,5.0,1.0,16:59:43,NaN,NaN,16
533,118.0,233.0,Is infanticide heavily biased against girls?,8,fdhja3,https://www.reddit.com/r/MensRights/comments/f...,12,1.583376e+09,You may know how female infanticide is notorio...,2020.0,3.0,5.0,02:41:15,NaN,NaN,None
534,119.0,237.0,Domestic Abuse Case in GA of USA,1,g9xwxd,https://www.reddit.com/r/legaladvice/comments/...,0,1.588144e+09,Hello. I want to help my mother fight against...,2020.0,4.0,29.0,07:10:06,NaN,NaN,15
535,120.0,239.0,Notarization confusion,1,fslkkw,https://www.reddit.com/r/legaladvice/comments/...,6,1.585718e+09,So I'm an international student from India and...,2020.0,4.0,1.0,05:18:40,NaN,NaN,10


In [53]:
df.drop(['Unnamed: 0','index','Unnamed: 14'],axis=1,inplace=True)

In [54]:
df

,title,score,id,url,comms_num,created,body,year,month,day,time,violence_type,age
0,There is NO ROOM for abuse.Period. Digital Pai...,3114,gfoc94,https://i.redd.it/xfvcce15rhx41.jpg,329,1.588951e+09,None,2020.0,5.0,8.0,15:12:12,abuse,None
1,"Behind closed doors, the biggest viruses are m...",1128,fzzzji,https://i.redd.it/dc1v110v0fs41.jpg,213,1.586738e+09,None,2020.0,4.0,13.0,00:40:34,abuse,None
2,"Manipur girl racially attacked, abused by loca...",365,gici3u,https://newsd.in/manipur-girl-racially-attacke...,62,1.589323e+09,None,2020.0,5.0,12.0,22:43:00,abuse,None
3,Chennai man caught on camera hurling casteist ...,402,g30ho5,https://www.thenewsminute.com/article/chennai-...,114,1.587153e+09,None,2020.0,4.0,17.0,19:49:59,abuse,None
5,Parental Abuse: What to do?,31,ggfu6u,https://www.reddit.com/r/india/comments/ggfu6u...,49,1.589061e+09,I am writing this with such a heavy heart. But...,2020.0,5.0,9.0,21:56:50,abuse,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Do the parties signing a legal contract have c...,1,gbeobj,https://www.reddit.com/r/legaladvice/comments/...,0,1.588352e+09,I am a 16 yo developer and I have an idea abou...,2020.0,5.0,1.0,16:59:43,NaN,16
533,Is infanticide heavily biased against girls?,8,fdhja3,https://www.reddit.com/r/MensRights/comments/f...,12,1.583376e+09,You may know how female infanticide is notorio...,2020.0,3.0,5.0,02:41:15,NaN,None
534,Domestic Abuse Case in GA of USA,1,g9xwxd,https://www.reddit.com/r/legaladvice/comments/...,0,1.588144e+09,Hello. I want to help my mother fight against...,2020.0,4.0,29.0,07:10:06,NaN,15
535,Notarization confusion,1,fslkkw,https://www.reddit.com/r/legaladvice/comments/...,6,1.585718e+09,So I'm an international student from India and...,2020.0,4.0,1.0,05:18:40,NaN,10


In [56]:
df.to_csv('reddit_domestic_violence.csv')